In [1]:
# Standard library imports
import os
import time
import shutil
import random
import math
import pickle
import gzip
import io
import PIL
import torch
import torchvision
import timm
import einops

# Third-party imports
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import fastcore.all as fc
from collections.abc import Mapping
from pathlib import Path
from operator import attrgetter, itemgetter
from functools import partial
from copy import copy
from contextlib import contextmanager

# PyTorch imports
from torch import nn, optim, tensor
from torch.nn import functional as F, init
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, default_collate
from torchvision import transforms
import torchvision.transforms.functional as TF

# miniai imports
from miniai.datasets import *
from miniai.conv import *
from miniai.learner import *
from miniai.activations import *
from miniai.init import *
from miniai.sgd import *
from miniai.resnet import *
from miniai import *

## Neural Cellular Automata

A cellular automaton is a discrete model that consists of a grid of cells, each of which can be in one of a finite number of states. The cells are updated simultaneously based on a set of rules that determine the state of a cell based on the states of its neighbors.

Cellular automata are often used to model complex systems and can exhibit emergent behavior, meaning that patterns and behaviors emerge from the interactions of the individual cells even though the rules governing their behavior are simple.

Classic examples such as the famous 'Game of Life' have very simple rules and limit states to 'alive' or 'dead'. However, the ideas can be extended to continuous outputs for each cell, and the update 'rule' can be a small neural network rather than a hard-coded decision tree - giving us 'Neural Cellular Automata'.

Here's what our NCA will look like:

![nca.png](images/nca.png)

Key references: 
- [Growing Neural Cellular Automata](https://distill.pub/2020/growing-ca/) - A delightful paper that was my intro to the topic

- [Texture Generation with NCA](https://arxiv.org/abs/2105.07299) - tiny models making amazing textures, the paper we're replicating today.

- ['The Future of Artificial Intelligence is Self-Organizing and Self-Assembling'](https://sebastianrisi.com/self_assembling_ai/) - More general discussion of this space

- [Fixing Neural CA Colors with Sliced Optimal Transport](https://www.youtube.com/watch?v=ZFYZFlY7lgI) -  A follow-on video from Alexander Mordvintsev (include code for a different style loss)

- [Fun with Neural Cellular Automata](https://wandb.ai/johnowhitaker/nca/reports/Fun-with-Neural-Cellular-Automata--VmlldzoyMDQ5Mjg0) - My W&B report with lots of examples